# Классификация текстов

В данном задании мы будем работать над задачей классификации текстов с использлванием различных методов векторизации слов.

In [1]:
from collections import Counter

import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score

import torch
from torch import nn
from torch.nn import functional as F
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau

from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline

out_dict = dict()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
# !pip install ...

### 1.1. Предобработка текста и токенизация

Для начала скачаем исходные данные:

In [3]:
# Считываем исходный набор данных
df = pd.read_csv( 
    'https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv',
    delimiter='\t', # разделитель - символ табуляции
    header=None     # заголовок отсутствует
)

In [4]:
# Посмотрим на первые 5 строк в считанном наборе данных
df.head(5)

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [5]:
len(df)

6920

In [6]:
df[0]

0       a stirring , funny and finally transporting re...
1       apparently reassembled from the cutting room f...
2       they presume their audience wo n't sit still f...
3       this is a visually stunning rumination on love...
4       jonathan parker 's bartleby should have been t...
                              ...                        
6915    painful , horrifying and oppressively tragic ,...
6916    take care is nicely performed by a quintet of ...
6917    the script covers huge , heavy topics in a bla...
6918    a seriously bad film with seriously warped log...
6919    a deliciously nonsensical comedy about a city ...
Name: 0, Length: 6920, dtype: object

Предобработка входных данных достаточно проста и подробно описывается в комментариях к коду.

Библиотека `nltk` широко используется при обработке текстов. 

In [7]:
# В качестве обучающей выборки выбираем первые 5000 предложений
texts_train = df[0].values[:5000]
y_train = df[1].values[:5000]

# В качестве тестовой выборки используем все оставшиеся предложения
texts_test = df[0].values[5000:]
y_test = df[1].values[5000:]

In [8]:
len(texts_train)

5000

In [9]:
y_train

array([1, 0, 0, ..., 1, 1, 1])

In [10]:
# В качестве токенов будем использовать отдельные слова и знаки препинания
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer() 

In [11]:
# Предобработанный текст будем представлять в виде выделенных токенов, 
# разделённых пробелом

preprocess = lambda text: ' '.join(tokenizer.tokenize(text.lower()))

In [12]:
# Посмотрим, как работает предобработка для заданной строки text

text = 'How to be a grown-up at work: replace "I don\'t want to do that" with "Ok, great!".'
print("before:", text,)
print("after:", preprocess(text),)

before: How to be a grown-up at work: replace "I don't want to do that" with "Ok, great!".
after: how to be a grown - up at work : replace " i don ' t want to do that " with " ok , great !".


In [13]:
preprocess(text)

'how to be a grown - up at work : replace " i don \' t want to do that " with " ok , great !".'

In [14]:
text

'How to be a grown-up at work: replace "I don\'t want to do that" with "Ok, great!".'

In [15]:
# Примените функцию preprocess к каждому тексту
texts_train = [preprocess(texts_train[i]) for i in range(len(texts_train))]
texts_test = [preprocess(text) for text in texts_test]

In [16]:
# Выполняем небольшие проверки того, насколько корректно были обработаны тренировочная и тестовая выборки
assert texts_train[5] ==  'campanella gets the tone just right funny in the middle of sad in the middle of hopeful'
assert texts_test[74] == 'poetry in motion captured on film'
assert len(texts_test) == len(y_test)

Следующие функции помогут вам с визуализацией процесса обучения сети.

In [17]:
#  Не изменяйте блок кода ниже!

# __________start of block__________
def plot_train_process(train_loss, val_loss, train_accuracy, val_accuracy, title_suffix=''):
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    axes[0].set_title(' '.join(['Loss', title_suffix]))
    axes[0].plot(train_loss, label='train')
    axes[0].plot(val_loss, label='validation')
    axes[0].legend()

    axes[1].set_title(' '.join(['Validation accuracy', title_suffix]))
    axes[1].plot(train_accuracy, label='train')
    axes[1].plot(val_accuracy, label='validation')
    axes[1].legend()
    plt.show()

def visualize_and_save_results(model, model_name, X_train, X_test, y_train, y_test, out_dict):
    for data_name, X, y, model in [
    ('train', X_train, y_train, model),
    ('test', X_test, y_test, model)
    ]:
        if isinstance(model, BaseEstimator):
            proba = model.predict_proba(X)[:, 1]
        elif isinstance(model, nn.Module):
            proba = model(X).detach().cpu().numpy()[:, 1]
        else:
            raise ValueError('Unrecognized model type')
            
        auc = roc_auc_score(y, proba)

        out_dict['{}_{}'.format(model_name, data_name)] = auc
        plt.plot(*roc_curve(y, proba)[:2], label='%s AUC=%.4f' % (data_name, auc))

    plt.plot([0, 1], [0, 1], '--', color='black',)
    plt.legend(fontsize='large')
    plt.title(model_name)
    plt.grid()
    return out_dict
# __________end of block__________

### 1.2. Мешок слов 

Текст -- последовательность токенов, длина которой не фиксирована. Соответственно, если каждый текст представлять матрицей, где каждый столбец будет являться векторным описанием очередного токена, мы будем получать для разных текстов матрицы разного размера. Как мы знаем, такие матрицы непригодны для передачи в нейронные сети, поскольку размер входных данных сети - всегда фиксирован. Поэтому нужно научиться представлять любой текст некоторым тензором (вектором или матрицей) фиксированного размера.

Самый простой способ -- воспользоваться классическим подходом к векторизации текстов: `мешком слов`. На лекции упоминалось, что в таком подходе каждый токен может быть закодирован разреженным вектором, размер которого будет равен размеру словаря. Соответственно, чтобы получить векторное представление всего текста, достаточно сложить векторные представления всех входящих в него токенов.

Первое, что мы рассмотрим - `One-hot` кодирование токенов. В этом случае мы приходим к тому, что текст будет представлен вектором, длина которого равна размеру словаря, N-ым значением в этом векторе будет количество употребления слова с индексом N в исходном тексте.

Для реализации такого подхода вы можете как воспользоваться `CountVectorizer` из `sklearn`, так и самостоятельно реализованный вариант. Обращаем ваше внимание, в данной задаче используется лишь k наиболее часто встречаемых слов из обучающей части выборки.

In [18]:
# Отбираем только k наиболее популярных слов в текстах
k = min(10000, len(set(' '.join(texts_train).split())))
print("k =", k)

k = 10000


In [55]:
# Построим словарь всех уникальных слов в обучающей выборке,
# оставив только k наиболее популярных слов.

counts = Counter(' '.join(texts_train).split()) 
bow_vocabulary = [key for key, val in counts.most_common(k)] # YOUR CODE

In [56]:
def text_to_bow(text):
    """ 
        Функция, позволяющая превратить входную строку 
        в векторное представление на основании модели мешка слов. 
    """
    # Создайте массив нулей размерности словаря 
    sent_vec = np.zeros((len(bow_vocabulary))) # 10000

    # Создайте словарь {word:count} в котором word - слово, count - встречаемость
    counts = Counter(text.split())

    # Положите в sent_vec соответствующие индексы значения counts
    # так чтобы получился BOW
    for i, token in enumerate(bow_vocabulary):
        if token in counts:
            sent_vec[i] = counts[token]
    bow = np.array(sent_vec, 'float32')
    
    return bow

In [57]:
# Векторизуйте ваши тексты в BOW с помощью выше написанной функции 
X_train_bow = np.stack(list(map(text_to_bow, texts_train))) #YOUR CODE
X_test_bow = np.stack(list(map(text_to_bow, texts_test))) #YOUR CODE

In [58]:
# Небольшие проверки
k_max = len(set(' '.join(texts_train).split()))
assert X_train_bow.shape == (len(texts_train), min(k, k_max))
assert X_test_bow.shape == (len(texts_test), min(k, k_max))
assert np.all(X_train_bow[5:10].sum(-1) == np.array([len(s.split()) for s in  texts_train[5:10]]))
assert len(bow_vocabulary) <= min(k, k_max)
assert X_train_bow[65, bow_vocabulary.index('!')] == texts_train[65].split().count('!')

In [59]:
X_train_bow 

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 2., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [3., 1., 1., ..., 0., 0., 0.]], dtype=float32)

In [61]:
# Строим модель лог регрессии для полученных векторных представлений текстов
from sklearn.linear_model import LogisticRegression

bow_model = LogisticRegression()

# Обучаем
bow_model(X_train_bow, y_train)

TypeError: 'LogisticRegression' object is not callable

* Что такое лог регрессия?

In [ ]:
# Визуализируем
out_dict = visualize_and_save_results(bow_model, 'bow_log_reg_sklearn', X_train_bow, X_test_bow, y_train, y_test, out_dict)

* Какие выводы можно сделать?

### 1.3. Логистическая регрессия в PyTorch

Сейчас же реализуйте решение на основе логистической регрессии, но уже используя PyTorch. В результате вам должна быть доступна обученная модель, предсказывающая вероятности для двух классов. Качество на тестовой выборке должно не уступать логистической регрессии.

Таким образом, нужно реализовать однослойную линейную сеть с использованием softmax в качестве функции активации и количеством выходов, равному количеству классов.

In [61]:
class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = # YOUR CODE

    def forward(self, x):
        outputs = F.softmax(# YOUR CODE, dim=-1) 
        return outputs

In [ ]:
# размер словаря
input_dim = # YOUR CODE

# количество классов 
output_dim = # YOUR CODE

print(f"input_dim = {input_dim}, output_dim = {output_dim}")

model = # YOUR CODE

Не забывайте о функциях потерь: `nn.CrossEntropyLoss` объединяет в себе `LogSoftMax` и `NLLLoss`. 

$$
\mathcal{L} = -\frac{1}{N} \sum_{i=1}^{N} \log\left( \frac{e^{x_{i, y_i}}}{\sum_{j=1}^{C} e^{x_{i, j}}} \right)
 -\frac{1}{N} \sum_{i=1}^{N} \left( x_{i, y_i} - \log \sum_{j=1}^{C} e^{x_{i, j}} \right)
$$


Также не забывайте о необходимости перенести тензоры на используемый `device`.

In [63]:
# Указываем лосс
loss_function = # YOUR CODE

# Какой оптимизатор используем?
opt = # YOUR CODE

In [64]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

X_train_bow_torch = torch.tensor(X_train_bow, dtype=torch.float32).to(device)
X_test_bow_torch = torch.tensor(X_test_bow, dtype=torch.float32).to(device)

y_train_torch = torch.tensor(y_train, dtype=torch.long).to(device)
y_test_torch = torch.tensor(y_test, dtype=torch.long).to(device)

Функция ниже поможет с обучением модели. Часть кода необходимо реализовать самостоятельно.

In [65]:
def train_model(
    model,
    opt,
    X_train_torch,
    y_train_torch,
    X_val_torch,
    y_val_torch,
    n_iterations=500,
    batch_size=32,
    show_plots=True,
    eval_every=50
):
    train_loss_history = []
    train_acc_history = []
    val_loss_history = []
    val_acc_history = []

    local_train_loss_history = []
    local_train_acc_history = []

    # Цикл по итерациям
    for # YOUR CODE

        # Получаем случайный батч размера batch_size для проведения обучения
        # Воспользуйтесь random.randint(low, high=None, size=None, dtype=int)
        ix = # YOUR CODE
        x_batch = # YOUR CODE
        y_batch = # YOUR CODE

        # Предсказываем отклик
        y_predicted = # YOUR CODE
        
        # Вычисляем loss, как и выше
        loss = # YOUR CODE

        # Вычисляем градиенты
        loss.backward() 

        # Adam step
        opt.step()

        # clear gradients
        opt.zero_grad()

        local_train_loss_history.append(loss.item())
        local_train_acc_history.append(
            accuracy_score(
                y_batch.to('cpu').detach().numpy(),
                y_predicted.to('cpu').detach().numpy().argmax(axis=1)
            )
        )

        # Делаем eval-шаг для каждой eval_every итерации
        if # YOUR CODE
            train_loss_history.append(np.mean(local_train_loss_history))
            train_acc_history.append(np.mean(local_train_acc_history))
            local_train_loss_history, local_train_acc_history = [], []

            # Предсказания
            predictions_val = # YOUR CODE
            val_loss_history.append(loss_function(predictions_val, y_val_torch).to('cpu').detach().item())

            acc_score_val = accuracy_score(y_val_torch.cpu().numpy(), predictions_val.to('cpu').detach().numpy().argmax(axis=1))
            val_acc_history.append(acc_score_val)

            if show_plots:
                display.clear_output(wait=True)
                plot_train_process(train_loss_history, val_loss_history, train_acc_history, val_acc_history)
    return model

In [ ]:
# Обучим на 3000 итераций
bow_nn_model = # YOUR CODE

In [ ]:
# Не изменяйте блок кода ниже!
# __________start of block__________
out_dict = visualize_and_save_results(bow_nn_model, 'bow_nn_torch', X_train_bow_torch, X_test_bow_torch, y_train, y_test, out_dict)

assert out_dict['bow_log_reg_sklearn_test'] - out_dict['bow_nn_torch_test'] < 0.01, 'AUC ROC on test data should be close to the sklearn implementation'
# __________end of block__________

А теперь повторите процедуру обучения выше, но для различных значений k – размера словаря. 

В список`results` сохраните `AUC ROC` на тестовой части выборки для модели, обученной со словарем размера k.

In [ ]:
vocab_sizes_list = np.arange(100, 5800, 700)
vocab_sizes_list

In [69]:
results = []

for k in vocab_sizes_list:
    
    # Отбор k наиболее популярных слов
    # YOUR CODE

    # Создание BOW
    # YOUR CODE

    # Переносим тензоры на device
    X_train_bow_torch = torch.tensor(X_train_bow, dtype=torch.float32).to(device)
    X_test_bow_torch = torch.tensor(X_test_bow, dtype=torch.float32).to(device)
    
    # Создание и обучение модели
    # YOUR CODE
    model = # YOUR CODE  # Заменяем input_dim на k
    opt = # YOUR CODE
    bow_nn_model = # YOUR CODE
    
    # Предсказание вероятностей для тестовой выборки
    predicted_probas_on_test = bow_nn_model(X_test_bow_torch).detach().cpu().numpy()
    auc = roc_auc_score(y_test, predicted_probas_on_test[:, 1])
    results.append(auc)

In [ ]:
# Не меняйте блок кода ниже!

# __________start of block__________
assert len(results) == len(vocab_sizes_list), 'Check the code above'
assert min(results) >= 0.65, 'Seems like the model is not trained well enough'
assert results[-1] > 0.84, 'Best AUC ROC should not be lower than 0.84'

plt.plot(vocab_sizes_list, results)
plt.xlabel('num of tokens')
plt.ylabel('AUC')
plt.grid()

out_dict['bow_k_vary'] = results
# __________end of block__________

### 1.4. Использование TF-iDF признаков.

Для векторизации текстов также можно воспользоваться `TF-iDF`. Это позволяет исключить из рассмотрения многие слова, не оказывающие значимого влияния при оценке непохожести текстов.

Ваша задача: векторизовать тексты используя TF-iDF (или TfidfVectorizer из sklearn, или реализовав его самостоятельно) и построить классификатор с помощью PyTorch.

Затем также оцените качество классификации по AUC ROC для различных размеров словаря.

Качество классификации должно быть не ниже 0.86 AUC ROC.

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(texts_train).toarray()
X_test_tfidf = vectorizer.transform(texts_test).toarray()

X_train_tfidf_torch = torch.tensor(X_train_tfidf, dtype=torch.float32).to(device)
X_test_tfidf_torch = torch.tensor(X_test_tfidf, dtype=torch.float32).to(device)

In [72]:
input_dim = X_train_tfidf.shape[1]
output_dim = 2
model = LogisticRegression(input_dim, output_dim).to(device)

In [ ]:
loss_function = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

model_tf_idf = train_model(model, opt, X_train_tfidf_torch, y_train_torch, X_test_tfidf_torch, y_test_torch, n_iterations=3000)

In [ ]:
# Не меняйте блок кода ниже!

# __________start of block__________
out_dict = visualize_and_save_results(model_tf_idf, 'tf_idf_nn_torch', X_train_tfidf_torch, X_test_tfidf_torch, y_train, y_test, out_dict)

assert out_dict['tf_idf_nn_torch_test'] >= out_dict['bow_nn_torch_test'], 'AUC ROC on test data should be better or close to BoW for TF-iDF features'
# __________end of block__________

In [75]:
vocab_sizes_list = np.arange(100, 5800, 700)
results = []

for k in vocab_sizes_list:

    # Векторизация текстов с помощью TF-IDF
    # YOUR CODE

    # Переносим тензоры на device
    X_train_tfidf_torch = torch.tensor(X_train_tfidf, dtype=torch.float32).to(device)
    X_test_tfidf_torch = torch.tensor(X_test_tfidf, dtype=torch.float32).to(device)

    # Создание и обучение модели
    model = # YOUR CODE
    opt = # YOUR CODE
    tf_idf_nn_model = # YOUR CODE

    # Предсказание вероятностей для тестовой выборки
    predicted_probas_on_test = tf_idf_nn_model(X_test_tfidf_torch).detach().cpu().numpy()
    auc = roc_auc_score(y_test, predicted_probas_on_test[:, 1])
    results.append(auc)

In [ ]:
# Не меняйте блок кода ниже!

# __________start of block__________
assert len(results) == len(vocab_sizes_list), 'Check the code above'
assert min(results) >= 0.65, 'Seems like the model is not trained well enough'
assert results[-1] > 0.85, 'Best AUC ROC for TF-iDF should not be lower than 0.84'

plt.plot(vocab_sizes_list, results)
plt.xlabel('num of tokens')
plt.ylabel('AUC')
plt.grid()

out_dict['tf_idf_k_vary'] = results
# __________end of block__________

### 1.5. Сравнение с Наивным Байесовским классификатором.

Классические модели все еще способны показать хороший результат во многих задачах. Обучите наивный байесовский классификатор на текстах, векторизованных с помощью BoW и TF-iDF и сравните результаты с моделями выше.

Комментарий: обращаем ваше внимание, необходимо выбрать подходящее к данной задаче априорное распределение для признаков, т.е. выбрать верную версию классификатора из sklearn: GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Обучение наивного байесовского классификатора на BoW
clf_nb_bow = MultinomialNB()
clf_nb_bow.fit(X_train_bow, y_train)

# do not change the code in the block below
# __________start of block__________
out_dict = visualize_and_save_results(clf_nb_bow, 'bow_nb_sklearn', X_train_bow, X_test_bow, y_train, y_test, out_dict)
# __________end of block__________

In [ ]:
clf_nb_tfidf = MultinomialNB()
clf_nb_tfidf.fit(X_train_tfidf, y_train)

# do not change the code in the block below
# __________start of block__________
out_dict = visualize_and_save_results(clf_nb_tfidf, 'tf_idf_nb_sklearn', X_train_tfidf, X_test_tfidf, y_train, y_test, out_dict)
# __________end of block__________

In [79]:
# do not change the code in the block below
# __________start of block__________
assert out_dict['tf_idf_nb_sklearn_test'] > out_dict['bow_nb_sklearn_test'],' TF-iDF results should be better'
assert out_dict['tf_idf_nb_sklearn_test'] > 0.86, 'TF-iDF Naive Bayes score should be above 0.86'
# __________end of block__________

### 1.6. Использование предобученных эмбеддингов

Наконец, воспользуемся предобученными эмбеддингами из библиотеки `gensim`. В нем доступно несколько эмбеддингов, предобученных на различных корпусах текстов. 

Напоминаем, что лучше использовать те эмбеддинги, которые были обучены на текстах похожей структуры.

Ваша задача: обучить модель (достаточно логистической регрессии или же двуслойной неронной сети), используя усредненный эмбеддинг для всех токенов в отзыве, добиться качества не хуже, чем с помощью `BoW`/`TF-iDF` и снизить степень переобучения (разницу между `AUC ROC` на обучающей и тестовой выборках).

**Обратите внимание!**

В Google Colab установлена устаревшая версия библиотеки gensim, поэтому некоторые её обновлённые возможности могут не поддерживаться.

Например, может возникать ошибка AttributeError.

В этом случае нужно раскомментировать строку ниже, выполнить данную ячейку и повторно выполнить все последующие ячейки:

In [80]:
# !pip install --upgrade gensim

In [ ]:
import gensim.downloader as api

gensim_embedding_model = api.load('word2vec-google-news-300')

In [82]:
def text_to_average_embedding(text, gensim_embedding_model):
    tokens = text.split()
    embeddings = [gensim_embedding_model[token] for token in tokens if token in gensim_embedding_model]
    if len(embeddings) > 0:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(gensim_embedding_model.vector_size)

In [83]:
X_train_emb = [text_to_average_embedding(text, gensim_embedding_model) for text in texts_train]
X_test_emb = [text_to_average_embedding(text, gensim_embedding_model) for text in texts_test]

assert len(X_train_emb[0]) == gensim_embedding_model.vector_size, 'Seems like the embedding shape is wrong'

In [ ]:
X_train_emb_torch = torch.tensor(X_train_emb, dtype=torch.float32)
X_test_emb_torch = torch.tensor(X_test_emb, dtype=torch.float32)

y_train_torch = torch.tensor(y_train, dtype=torch.long)
y_test_torch = torch.tensor(y_test, dtype=torch.long)

In [85]:
model = nn.Sequential(
    nn.Linear(gensim_embedding_model.vector_size, 2)
)

In [ ]:
loss_function = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

model = train_model(model, opt, X_train_emb_torch, y_train_torch, X_test_emb_torch, y_test_torch, n_iterations=3000)

In [ ]:
# do not change the code in the block below
# __________start of block__________

out_dict = visualize_and_save_results(model, 'emb_nn_torch', X_train_emb_torch, X_test_emb_torch, y_train, y_test, out_dict)
assert out_dict['emb_nn_torch_test'] > 0.87, 'AUC ROC on test data should be better than 0.86'
assert out_dict['emb_nn_torch_train'] - out_dict['emb_nn_torch_test'] < 0.1, 'AUC ROC on test and train data should not be different more than by 0.1'
# __________end of block__________